In [ ]:
import streamlit as st
import pandas as pd
import phonenumbers
import json
import datetime
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import (Mail, Attachment, FileContent, FileName, FileType, Disposition, ContentId, To)
import base64

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
-- Get sendgrid_api_token from Snowflake secrets
use schema PYTHON_WORKLOADS.email_reports;
CREATE OR REPLACE function sendgrid_api_key_secret()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.9
HANDLER = 'get_secret_1'
EXTERNAL_ACCESS_INTEGRATIONS = (SENDGRID_APIS_ACCESS_INTEGRATION_EMAIL)
SECRETS = ('cred' = sendgrid_dev_API_TOKEN )
AS
$$
import _snowflake
def get_secret_1():
   secret_type = _snowflake.get_generic_secret_string('cred')
   return secret_type
$$;
grant usage on function sendgrid_api_key_secret() to role svc_engineer_prd;

In [ ]:
# Store sf api token in variable
api_secret_query = "select PYTHON_WORKLOADS.email_reports.sendgrid_api_key_secret()"
sendgrid_api_key = session.sql(api_secret_query).collect()[0][0]

In [ ]:
insert into bi.marketing.intent_campaign (
select  
b.ZI_C_LOCATION_ID,
b.ZI_C_COMPANY_ID,
a.ZI_C_COMPANY_NAME,
b.ZI_C_STREET,
b.ZI_C_STREET_2,
b.ZI_C_CITY,
b.ZI_C_STATE,
b.ZI_C_ZIP,
b.ZI_C_COUNTRY,
b.ZI_ES_LOCATION_ID,
b.ZI_C_URL,
b.ZI_C_LATITUDE,
b.ZI_C_LONGITUDE,
b.ZI_C_EMPLOYEES,
b.ZI_C_REVENUE_RANGE,
b.ZI_C_REVENUE,
coalesce(b.Mobile_phone,b.Direct_Phone_Number,b.ZI_C_COMPANY_PHONE,b.ZI_C_PHONE) as phone,
b.ZI_C_INDUSTRY_PRIMARY,
b.ZI_C_SUB_INDUSTRY_PRIMARY,
b.ZI_C_SIC4,
b.ZI_C_NAICS4,
b.ZI_C_COMPANY_REVENUE_RANGE,
b.ZI_C_LINKEDIN_URL,
b.ZI_C_FACEBOOK_URL,
b.ZI_C_TWITTER_URL,
b.ZI_C_YELP_URL,
b.ZI_C_EIN,
b.ZI_C_NUM_LOCATIONS,
b.ZI_C_HR_CONTACTS,
b.ZI_C_SALES_CONTACTS,
b.ZI_C_MARKETING_CONTACTS,
b.ZI_C_FINANCE_CONTACTS,
b.ZI_C_C_SUITE_CONTACTS,
b.ZI_C_ENGINEERING_CONTACTS,
b.ZI_C_IT_CONTACTS,
b.ZI_C_OPERATIONS_CONTACTS,
b.ZI_C_TECH_IDS,
b.ZoomInfo_Contact_ID,
b.Last_Name,
b.First_Name,
b.Middle_Name,
b.Suffix,
b.Job_Title,
b.Job_Title_Hierarchy_Level,
b.Job_Function,
b.Management_Level,
b.Company_Division_Name,
coalesce(b.Email_Address,b.Supplemental_Email) as email,
b.Email_Domain,
b.Department,
b.Contact_Accuracy_Score,
b.Contact_Accuracy_Grade,
b.ZoomInfo_Contact_Profile_URL,
b.LinkedIn_Contact_Profile_URL,
b.Notice_Provided_Date,
b.Person_Street,
b.Person_City,
b.Person_State,
b.Person_Zip_Code,
b.Country,
b.ZoomInfo_Company_ID,
a.INTENT_ID,
a.DATE,
a.TOPIC,
a.CATEGORY,
a.SIGNAL_SCORE,
a.AUDIENCE_STRENGTH,
a.SIGNAL_LOCATION,
a.ZI_C_COMPANY_REVENUE,
'Payment' as product,
current_date as dtecreated,
b.Mobile_phone as mobile
from (select * from zoominfo_rw_secondary.zoominfo.zi_intent_company_data where 
date>CURRENT_DATE()-7  and not contains(lower(ZI_C_COMPANY_NAME),'library') and not contains(lower(ZI_C_COMPANY_NAME),'museum')
and ZI_C_COMPANY_PRIMARY_INDUSTRY != 'Retail' and ZI_C_COMPANY_REVENUE >= 250 and ZI_C_COMPANY_REVENUE <= 25000) a
left join (select * from zoominfo.brick.zi_company_contact_combined where ZoomInfo_Contact_ID is not null and ZI_C_REVENUE_RANGE in ('$0M-1M','$1M-$5M','$5M-$10M','$10M-$25M') and country in ('Canada', 'United States') and Contact_Accuracy_Score > 80 and ZI_C_INDUSTRY_PRIMARY!='Retail') b ON b.ZI_C_COMPANY_ID = a.ZI_C_COMPANY_ID
join REFERENCE.MARKETING.ZI_TOPIC_PRODUCT_MAPPING c on a.topic=c.topic
where  (email not in (select distinct IFNULL(email,'NA') from bi.marketing.intent_campaign))
and ((Phone is not null or mobile is not null) and (Phone!='' or mobile is not null))
and not contains (email,'.gov') and not contains (email,'.edu')  and c.product='Payment'
and not exists (select 1 from REFERENCE.MARKETING.ZI_JOB_EXCLUSION_LIST where lower(b.job_title) like lower(job_title)|| '%')
qualify row_number() over (partition by b.ZoomInfo_Contact_ID order by a.SIGNAL_SCORE) = 1
and row_number() over (partition by phone order by Job_Title_Hierarchy_Level desc) = 1
-- and row_number() over (partition by b.ZoomInfo_Company_ID order by a.ZI_C_COMPANY_ID) <= 5
);

In [ ]:
insert into bi.marketing.intent_campaign (
select  
b.ZI_C_LOCATION_ID,
b.ZI_C_COMPANY_ID,
a.ZI_C_COMPANY_NAME,
b.ZI_C_STREET,
b.ZI_C_STREET_2,
b.ZI_C_CITY,
b.ZI_C_STATE,
b.ZI_C_ZIP,
b.ZI_C_COUNTRY,
b.ZI_ES_LOCATION_ID,
b.ZI_C_URL,
b.ZI_C_LATITUDE,
b.ZI_C_LONGITUDE,
b.ZI_C_EMPLOYEES,
b.ZI_C_REVENUE_RANGE,
b.ZI_C_REVENUE,
coalesce(b.Mobile_phone,b.Direct_Phone_Number,b.ZI_C_COMPANY_PHONE,b.ZI_C_PHONE) as phone,
b.ZI_C_INDUSTRY_PRIMARY,
b.ZI_C_SUB_INDUSTRY_PRIMARY,
b.ZI_C_SIC4,
b.ZI_C_NAICS4,
b.ZI_C_COMPANY_REVENUE_RANGE,
b.ZI_C_LINKEDIN_URL,
b.ZI_C_FACEBOOK_URL,
b.ZI_C_TWITTER_URL,
b.ZI_C_YELP_URL,
b.ZI_C_EIN,
b.ZI_C_NUM_LOCATIONS,
b.ZI_C_HR_CONTACTS,
b.ZI_C_SALES_CONTACTS,
b.ZI_C_MARKETING_CONTACTS,
b.ZI_C_FINANCE_CONTACTS,
b.ZI_C_C_SUITE_CONTACTS,
b.ZI_C_ENGINEERING_CONTACTS,
b.ZI_C_IT_CONTACTS,
b.ZI_C_OPERATIONS_CONTACTS,
b.ZI_C_TECH_IDS,
b.ZoomInfo_Contact_ID,
b.Last_Name,
b.First_Name,
b.Middle_Name,
b.Suffix,
b.Job_Title,
b.Job_Title_Hierarchy_Level,
b.Job_Function,
b.Management_Level,
b.Company_Division_Name,
coalesce(b.Email_Address,b.Supplemental_Email) as email,
b.Email_Domain,
b.Department,
b.Contact_Accuracy_Score,
b.Contact_Accuracy_Grade,
b.ZoomInfo_Contact_Profile_URL,
b.LinkedIn_Contact_Profile_URL,
b.Notice_Provided_Date,
b.Person_Street,
b.Person_City,
b.Person_State,
b.Person_Zip_Code,
b.Country,
b.ZoomInfo_Company_ID,
a.INTENT_ID,
a.DATE,
a.TOPIC,
a.CATEGORY,
a.SIGNAL_SCORE,
a.AUDIENCE_STRENGTH,
a.SIGNAL_LOCATION,
a.ZI_C_COMPANY_REVENUE,
'Payroll' as product,
current_date as dtecreated,
b.Mobile_phone as mobile
from (select * from zoominfo_rw_secondary.zoominfo.zi_intent_company_data where  date>CURRENT_DATE()-7
and ZI_C_COMPANY_PRIMARY_INDUSTRY != 'Retail' and ZI_C_COMPANY_REVENUE >= 250 and ZI_C_COMPANY_REVENUE <= 25000) a
left join (select * from zoominfo.brick.zi_company_contact_combined where ZoomInfo_Contact_ID is not null and ZI_C_REVENUE_RANGE in ('$0M-$1M','$1M-$5M','$5M-$10M','$10M-$25M')  and ZI_C_EMPLOYEES >4 and ZI_C_EMPLOYEES < 251 and country in ('United States') and Contact_Accuracy_Score > 80 and ZI_C_INDUSTRY_PRIMARY!='Retail'
) b ON b.ZI_C_COMPANY_ID = a.ZI_C_COMPANY_ID
join REFERENCE.MARKETING.ZI_TOPIC_PRODUCT_MAPPING c on a.topic=c.topic
where  (email not in (select distinct IFNULL(email,'NA') from bi.marketing.intent_campaign))
and ((Phone is not null or mobile is not null) and (Phone!='' or mobile is not null))
and not contains (email,'.gov') and not contains (email,'.edu')  and c.product='Payroll'
and not exists (select 1 from REFERENCE.MARKETING.ZI_JOB_EXCLUSION_LIST where lower(b.job_title) like lower(job_title)|| '%')
qualify row_number() over (partition by b.ZoomInfo_Contact_ID order by a.SIGNAL_SCORE) = 1
and row_number() over (partition by b.ZI_C_LOCATION_ID order by b.ZI_C_LOCATION_ID) <= 5
and row_number() over (partition by phone order by Job_Title_Hierarchy_Level desc) = 1
);


In [ ]:
insert into bi.marketing.intent_campaign (
select  
b.ZI_C_LOCATION_ID,
b.ZI_C_COMPANY_ID,
a.ZI_C_COMPANY_NAME,
b.ZI_C_STREET,
b.ZI_C_STREET_2,
b.ZI_C_CITY,
b.ZI_C_STATE,
b.ZI_C_ZIP,
b.ZI_C_COUNTRY,
b.ZI_ES_LOCATION_ID,
b.ZI_C_URL,
b.ZI_C_LATITUDE,
b.ZI_C_LONGITUDE,
b.ZI_C_EMPLOYEES,
b.ZI_C_REVENUE_RANGE,
b.ZI_C_REVENUE,
coalesce(b.Mobile_phone,b.Direct_Phone_Number,b.ZI_C_COMPANY_PHONE,b.ZI_C_PHONE) as phone,
b.ZI_C_INDUSTRY_PRIMARY,
b.ZI_C_SUB_INDUSTRY_PRIMARY,
b.ZI_C_SIC4,
b.ZI_C_NAICS4,
b.ZI_C_COMPANY_REVENUE_RANGE,
b.ZI_C_LINKEDIN_URL,
b.ZI_C_FACEBOOK_URL,
b.ZI_C_TWITTER_URL,
b.ZI_C_YELP_URL,
b.ZI_C_EIN,
b.ZI_C_NUM_LOCATIONS,
b.ZI_C_HR_CONTACTS,
b.ZI_C_SALES_CONTACTS,
b.ZI_C_MARKETING_CONTACTS,
b.ZI_C_FINANCE_CONTACTS,
b.ZI_C_C_SUITE_CONTACTS,
b.ZI_C_ENGINEERING_CONTACTS,
b.ZI_C_IT_CONTACTS,
b.ZI_C_OPERATIONS_CONTACTS,
b.ZI_C_TECH_IDS,
b.ZoomInfo_Contact_ID,
b.Last_Name,
b.First_Name,
b.Middle_Name,
b.Suffix,
b.Job_Title,
b.Job_Title_Hierarchy_Level,
b.Job_Function,
b.Management_Level,
b.Company_Division_Name,
coalesce(b.Email_Address,b.Supplemental_Email) as email,
b.Email_Domain,
b.Department,
b.Contact_Accuracy_Score,
b.Contact_Accuracy_Grade,
b.ZoomInfo_Contact_Profile_URL,
b.LinkedIn_Contact_Profile_URL,
b.Notice_Provided_Date,
b.Person_Street,
b.Person_City,
b.Person_State,
b.Person_Zip_Code,
b.Country,
b.ZoomInfo_Company_ID,
a.INTENT_ID,
a.DATE,
a.TOPIC,
a.CATEGORY,
a.SIGNAL_SCORE,
a.AUDIENCE_STRENGTH,
a.SIGNAL_LOCATION,
a.ZI_C_COMPANY_REVENUE,
'B2B' as product,
current_date as dtecreated,
b.Mobile_phone as mobile
from (select * from zoominfo_rw_secondary.zoominfo.zi_intent_company_data where  date>CURRENT_DATE()-7
and ZI_C_COMPANY_PRIMARY_INDUSTRY != 'Retail'
and ZI_C_COMPANY_PRIMARY_INDUSTRY != 'Construction' and ZI_C_COMPANY_REVENUE >= 100 and ZI_C_COMPANY_REVENUE <= 50000) a
left join (select * from zoominfo.brick.zi_company_contact_combined where ZoomInfo_Contact_ID is not null and ZI_C_EMPLOYEES <= 250 and ZI_C_REVENUE_RANGE in ('$0M-$1M','$1M-$5M','$5M-$10M','$10M-$25M','$25M-$50M') and country in ('Canada', 'United States') and Contact_Accuracy_Score > 80 and ZI_C_INDUSTRY_PRIMARY!='Retail' and ZI_C_INDUSTRY_PRIMARY!='Construction'
) b ON b.ZI_C_COMPANY_ID = a.ZI_C_COMPANY_ID
join REFERENCE.MARKETING.ZI_TOPIC_PRODUCT_MAPPING c on a.topic=c.topic
where  (email not in (select distinct IFNULL(email,'NA') from bi.marketing.intent_campaign))
and ((Phone is not null or mobile is not null) and (Phone!='' or mobile is not null))
and not contains (email,'.gov') and not contains (email,'.edu')  and c.product='B2B'
and not exists (select 1 from REFERENCE.MARKETING.ZI_JOB_EXCLUSION_LIST where lower(b.job_title) like lower(job_title)|| '%')
qualify row_number() over (partition by phone order by Job_Title_Hierarchy_Level desc) = 1
and row_number() over (partition by b.ZoomInfo_Contact_ID order by a.SIGNAL_SCORE) = 1
-- and row_number() over (partition by b.ZoomInfo_Company_ID order by a.ZI_C_COMPANY_ID) <= 5
);


In [ ]:
insert into bi.marketing.intent_campaign (
select  
b.ZI_C_LOCATION_ID,
b.ZI_C_COMPANY_ID,
a.ZI_C_COMPANY_NAME,
b.ZI_C_STREET,
b.ZI_C_STREET_2,
b.ZI_C_CITY,
b.ZI_C_STATE,
b.ZI_C_ZIP,
b.ZI_C_COUNTRY,
b.ZI_ES_LOCATION_ID,
b.ZI_C_URL,
b.ZI_C_LATITUDE,
b.ZI_C_LONGITUDE,
b.ZI_C_EMPLOYEES,
b.ZI_C_REVENUE_RANGE,
b.ZI_C_REVENUE,
coalesce(b.Mobile_phone,b.Direct_Phone_Number,b.ZI_C_COMPANY_PHONE,b.ZI_C_PHONE) as phone,
b.ZI_C_INDUSTRY_PRIMARY,
b.ZI_C_SUB_INDUSTRY_PRIMARY,
b.ZI_C_SIC4,
b.ZI_C_NAICS4,
b.ZI_C_COMPANY_REVENUE_RANGE,
b.ZI_C_LINKEDIN_URL,
b.ZI_C_FACEBOOK_URL,
b.ZI_C_TWITTER_URL,
b.ZI_C_YELP_URL,
b.ZI_C_EIN,
b.ZI_C_NUM_LOCATIONS,
b.ZI_C_HR_CONTACTS,
b.ZI_C_SALES_CONTACTS,
b.ZI_C_MARKETING_CONTACTS,
b.ZI_C_FINANCE_CONTACTS,
b.ZI_C_C_SUITE_CONTACTS,
b.ZI_C_ENGINEERING_CONTACTS,
b.ZI_C_IT_CONTACTS,
b.ZI_C_OPERATIONS_CONTACTS,
b.ZI_C_TECH_IDS,
b.ZoomInfo_Contact_ID,
b.Last_Name,
b.First_Name,
b.Middle_Name,
b.Suffix,
b.Job_Title,
b.Job_Title_Hierarchy_Level,
b.Job_Function,
b.Management_Level,
b.Company_Division_Name,
coalesce(b.Email_Address,b.Supplemental_Email) as email,
b.Email_Domain,
b.Department,
b.Contact_Accuracy_Score,
b.Contact_Accuracy_Grade,
b.ZoomInfo_Contact_Profile_URL,
b.LinkedIn_Contact_Profile_URL,
b.Notice_Provided_Date,
b.Person_Street,
b.Person_City,
b.Person_State,
b.Person_Zip_Code,
b.Country,
b.ZoomInfo_Company_ID,
a.INTENT_ID,
a.DATE,
a.TOPIC,
a.CATEGORY,
a.SIGNAL_SCORE,
a.AUDIENCE_STRENGTH,
a.SIGNAL_LOCATION,
a.ZI_C_COMPANY_REVENUE,
'B2B-ERP' as product,
current_date as dtecreated,
b.Mobile_phone as mobile
from (select * from zoominfo_rw_secondary.zoominfo.zi_intent_company_data where  date>CURRENT_DATE()-7
and ZI_C_COMPANY_PRIMARY_INDUSTRY != 'Retail'
and ZI_C_COMPANY_PRIMARY_INDUSTRY != 'Construction' and ZI_C_COMPANY_REVENUE >= 100 and category != 'Construction') a
left join (select * from zoominfo.brick.zi_company_contact_combined where ZoomInfo_Contact_ID is not null and ZI_C_EMPLOYEES > 50 and ZI_C_REVENUE_RANGE in ('$0M-$1M','$1M-$5M','$5M-$10M','$10M-$25M','$25M-$50M','$50M-$100M','$100M-$250M','$250M-$500M','$500M-$1B')  and country in ('Canada', 'United States') and Contact_Accuracy_Score > 80 and ZI_C_INDUSTRY_PRIMARY!='Retail' and ZI_C_INDUSTRY_PRIMARY!='Construction') b ON b.ZI_C_COMPANY_ID = a.ZI_C_COMPANY_ID
join REFERENCE.MARKETING.ZI_TOPIC_PRODUCT_MAPPING c on a.topic=c.topic
where  (email not in (select distinct IFNULL(email,'NA') from bi.marketing.intent_campaign))
and ((Phone is not null or mobile is not null) and (Phone!='' or mobile is not null))
and not contains (email,'.gov') and not contains (email,'.edu')  and c.product='B2B ERP'
and not exists (select 1 from REFERENCE.MARKETING.ZI_JOB_EXCLUSION_LIST where lower(b.job_title) like lower(job_title)|| '%')
qualify row_number() over (partition by b.ZoomInfo_Contact_ID order by a.SIGNAL_SCORE) = 1
and row_number() over (partition by phone order by Job_Title_Hierarchy_Level desc) = 1
-- and row_number() over (partition by b.ZoomInfo_Company_ID order by a.ZI_C_COMPANY_ID) <= 5
);

In [ ]:
insert into bi.marketing.intent_campaign (
select  
b.ZI_C_LOCATION_ID,
b.ZI_C_COMPANY_ID,
a.ZI_C_COMPANY_NAME,
b.ZI_C_STREET,
b.ZI_C_STREET_2,
b.ZI_C_CITY,
b.ZI_C_STATE,
b.ZI_C_ZIP,
b.ZI_C_COUNTRY,
b.ZI_ES_LOCATION_ID,
b.ZI_C_URL,
b.ZI_C_LATITUDE,
b.ZI_C_LONGITUDE,
b.ZI_C_EMPLOYEES,
b.ZI_C_REVENUE_RANGE,
b.ZI_C_REVENUE,
coalesce(b.Mobile_phone,b.Direct_Phone_Number,b.ZI_C_COMPANY_PHONE,b.ZI_C_PHONE) as phone,
b.ZI_C_INDUSTRY_PRIMARY,
b.ZI_C_SUB_INDUSTRY_PRIMARY,
b.ZI_C_SIC4,
b.ZI_C_NAICS4,
b.ZI_C_COMPANY_REVENUE_RANGE,
b.ZI_C_LINKEDIN_URL,
b.ZI_C_FACEBOOK_URL,
b.ZI_C_TWITTER_URL,
b.ZI_C_YELP_URL,
b.ZI_C_EIN,
b.ZI_C_NUM_LOCATIONS,
b.ZI_C_HR_CONTACTS,
b.ZI_C_SALES_CONTACTS,
b.ZI_C_MARKETING_CONTACTS,
b.ZI_C_FINANCE_CONTACTS,
b.ZI_C_C_SUITE_CONTACTS,
b.ZI_C_ENGINEERING_CONTACTS,
b.ZI_C_IT_CONTACTS,
b.ZI_C_OPERATIONS_CONTACTS,
b.ZI_C_TECH_IDS,
b.ZoomInfo_Contact_ID,
b.Last_Name,
b.First_Name,
b.Middle_Name,
b.Suffix,
b.Job_Title,
b.Job_Title_Hierarchy_Level,
b.Job_Function,
b.Management_Level,
b.Company_Division_Name,
coalesce(b.Email_Address,b.Supplemental_Email) as email,
b.Email_Domain,
b.Department,
b.Contact_Accuracy_Score,
b.Contact_Accuracy_Grade,
b.ZoomInfo_Contact_Profile_URL,
b.LinkedIn_Contact_Profile_URL,
b.Notice_Provided_Date,
b.Person_Street,
b.Person_City,
b.Person_State,
b.Person_Zip_Code,
b.Country,
b.ZoomInfo_Company_ID,
a.INTENT_ID,
a.DATE,
a.TOPIC,
a.CATEGORY,
a.SIGNAL_SCORE,
a.AUDIENCE_STRENGTH,
a.SIGNAL_LOCATION,
a.ZI_C_COMPANY_REVENUE,
'Online Payments' as product,
current_date as dtecreated,
b.Mobile_phone as mobile
from (select * from zoominfo_rw_secondary.zoominfo.zi_intent_company_data where  date>CURRENT_DATE()-7 and ZI_C_COMPANY_REVENUE >= 250 and not contains(lower(ZI_C_COMPANY_NAME),'restaurant') and not contains(lower(ZI_C_COMPANY_NAME),'college') and not contains(lower(ZI_C_COMPANY_NAME),'institute') and not contains(lower(ZI_C_COMPANY_NAME),'school') and ZI_C_COMPANY_REVENUE <= 30000) a
left join (select * from zoominfo.brick.zi_company_contact_combined where ZoomInfo_Contact_ID is not null and  ZI_C_REVENUE_RANGE in ('$0M-$1M','$1M-$5M','$5M-$10M','$10M-$25M','$25M-$50M') and country in ('Canada', 'United States') and  ZI_C_INDUSTRY_PRIMARY not in ('Transportation','Education','Construction','Holding Companies & Conglomerates','Healthcare Services','Government','Manufacturing','Law Firms & Legal Services','Insurance','Hospitals & Physicians Clinics','Minerals & Mining','Organizations','Real Estate') and not (ZI_C_INDUSTRY_PRIMARY='Telecommunications' and ZI_C_SUB_INDUSTRY_PRIMARY in ('Research & Development','Airlines, Airports & Air Services','Architecture, Engineering & Design','Government','Sports Teams & Leagues','Religious Organizations','Electricity, Oil & Gas','Lending & Brokerage','Venture Capital & Private Equity','Newspapers & News Services','Industrial Machinery & Equipment','Convenience Stores, Gas Stations & Liquor Stores','Elderly Care Services','Management Consulting','Freight & Logistics Services','Civil Engineering Construction','HR & Staffing','Building Materials','Training','Data Collection & Internet Portals','Mental Health & Rehabilitation Facilities','Medical Devices & Equipment','Office Products Retail & Distribution','Restaurants','Food & Beverage')) and not (ZI_C_INDUSTRY_PRIMARY='Software' and ZI_C_SUB_INDUSTRY_PRIMARY in ('Broadcasting','Civil Engineering Construction','Medical Laboratories & Imaging Centers','Investment Banking','Lending & Brokerage','Venture Capital & Private Equity','Enterprise Resource Planning (ERP)','Software','Aerospace & Defense','Freight & Logistics Services')) and not (ZI_C_INDUSTRY_PRIMARY='Retail' and ZI_C_SUB_INDUSTRY_PRIMARY in ('Industrial Machinery & Equipment','Animals & Livestock','Marine Shipping & Transportation','Convenience Stores, Gas Stations & Liquor Stores','Venture Capital & Private Equity','Music Production & Services','Boats & Submarines','Research & Development','Medical Devices & Equipment','Training','Rail, Bus & Taxi','Chemicals & Related Products','Government','Lending & Brokerage','Water Treatment','Management Consulting','Civil Engineering Construction','Pharmaceuticals','Architecture, Engineering & Design','Auctions','Oil & Gas Exploration & Services','Test & Measurement Equipment','Freight & Logistics Services','Medical Laboratories & Imaging Centers','Electricity, Oil & Gas','Facilities Management & Commercial Cleaning','Call Centers & Business Centers','Credit Cards & Transaction Processing','Automobile Dealers','Trucking, Moving & Storage','Airlines, Airports & Air Services','Elderly Care Services','HR & Staffing')) and not (ZI_C_INDUSTRY_PRIMARY='Business Services' and ZI_C_SUB_INDUSTRY_PRIMARY in ('Libraries','Training','Automobile Dealers','Debt Collection','Boats & Submarines','Test & Measurement Equipment','Oil & Gas Exploration & Services','Drug Stores & Pharmacies','Crops','Chambers of Commerce','Medical Laboratories & Imaging Centers','Freight & Logistics Services','Sports Teams & Leagues','Music Production & Services','Blood & Organ Banks','Investment Banking','Federal','Rail, Bus & Taxi','Building Materials','HR & Staffing','Broadcasting','Banking','Medical Devices & Equipment','Research & Development','Civil Engineering Construction','Lending & Brokerage','Electricity, Oil & Gas','Water Treatment','Forestry','Chemicals & Related Products','Minerals & Mining','Mental Health & Rehabilitation Facilities','Government','Venture Capital & Private Equity','Medical & Surgical Hospitals','Industrial Machinery & Equipment','Pulp & Paper','Convenience Stores, Gas Stations & Liquor Stores','Aerospace & Defense')) and not (ZI_C_INDUSTRY_PRIMARY='Agriculture' and ZI_C_SUB_INDUSTRY_PRIMARY in ('Auctions','Pharmaceuticals','Water Treatment','Convenience Stores, Gas Stations & Liquor Stores','Travel Agencies & Services','Lodging & Resorts','Elderly Care Services','Forestry','Rail, Bus & Taxi','Research & Development','Sports Teams & Leagues','Lending & Brokerage','Publishing','Computer Equipment & Peripherals','Wire & Cable','HR & Staffing','Mental Health & Rehabilitation Facilities','Animals & Livestock','Industrial Machinery & Equipment','Restaurants','Freight & Logistics Services','Chemicals & Related Products','Civil Engineering Construction','Amusement Parks, Arcades & Attractions','Architecture, Engineering & Design','Training')) and Contact_Accuracy_Score > 80) b ON b.ZI_C_COMPANY_ID = a.ZI_C_COMPANY_ID
join REFERENCE.MARKETING.ZI_TOPIC_PRODUCT_MAPPING c on a.topic=c.topic
where  email not in (select distinct IFNULL(email,'NA') from bi.marketing.intent_campaign)
and email not in (select distinct email from bi.marketing.intent_campaign where product='Payment')
and not contains (email,'.gov') and not contains (email,'.edu') and c.product='Online Payment'
and not exists (select 1 from REFERENCE.MARKETING.ZI_JOB_EXCLUSION_LIST where lower(b.job_title) like lower(job_title)|| '%')
and ((Phone is not null or mobile is not null) and (Phone!='' or mobile is not null))
qualify row_number() over (partition by b.ZoomInfo_Contact_ID order by a.SIGNAL_SCORE) = 1
and row_number() over (partition by phone order by Job_Title_Hierarchy_Level desc) = 1
);

In [ ]:
insert into bi.marketing.intent_campaign (
select  
b.ZI_C_LOCATION_ID,
b.ZI_C_COMPANY_ID,
a.ZI_C_COMPANY_NAME,
b.ZI_C_STREET,
b.ZI_C_STREET_2,
b.ZI_C_CITY,
b.ZI_C_STATE,
b.ZI_C_ZIP,
b.ZI_C_COUNTRY,
b.ZI_ES_LOCATION_ID,
b.ZI_C_URL,
b.ZI_C_LATITUDE,
b.ZI_C_LONGITUDE,
b.ZI_C_EMPLOYEES,
b.ZI_C_REVENUE_RANGE,
b.ZI_C_REVENUE,
coalesce(b.Mobile_phone,b.Direct_Phone_Number,b.ZI_C_COMPANY_PHONE,b.ZI_C_PHONE) as phone,
b.ZI_C_INDUSTRY_PRIMARY,
b.ZI_C_SUB_INDUSTRY_PRIMARY,
b.ZI_C_SIC4,
b.ZI_C_NAICS4,
b.ZI_C_COMPANY_REVENUE_RANGE,
b.ZI_C_LINKEDIN_URL,
b.ZI_C_FACEBOOK_URL,
b.ZI_C_TWITTER_URL,
b.ZI_C_YELP_URL,
b.ZI_C_EIN,
b.ZI_C_NUM_LOCATIONS,
b.ZI_C_HR_CONTACTS,
b.ZI_C_SALES_CONTACTS,
b.ZI_C_MARKETING_CONTACTS,
b.ZI_C_FINANCE_CONTACTS,
b.ZI_C_C_SUITE_CONTACTS,
b.ZI_C_ENGINEERING_CONTACTS,
b.ZI_C_IT_CONTACTS,
b.ZI_C_OPERATIONS_CONTACTS,
b.ZI_C_TECH_IDS,
b.ZoomInfo_Contact_ID,
b.Last_Name,
b.First_Name,
b.Middle_Name,
b.Suffix,
b.Job_Title,
b.Job_Title_Hierarchy_Level,
b.Job_Function,
b.Management_Level,
b.Company_Division_Name,
coalesce(b.Email_Address,b.Supplemental_Email) as email,
b.Email_Domain,
b.Department,
b.Contact_Accuracy_Score,
b.Contact_Accuracy_Grade,
b.ZoomInfo_Contact_Profile_URL,
b.LinkedIn_Contact_Profile_URL,
b.Notice_Provided_Date,
b.Person_Street,
b.Person_City,
b.Person_State,
b.Person_Zip_Code,
b.Country,
b.ZoomInfo_Company_ID,
a.INTENT_ID,
a.DATE,
a.TOPIC,
a.CATEGORY,
a.SIGNAL_SCORE,
a.AUDIENCE_STRENGTH,
a.SIGNAL_LOCATION,
a.ZI_C_COMPANY_REVENUE,
'Retail' as product,
current_date as dtecreated,
b.Mobile_phone as mobile
from (select * from zoominfo_rw_secondary.zoominfo.zi_intent_company_data where  date>CURRENT_DATE()-7  and lower(ZI_C_COMPANY_NAME) not regexp '.*(high school|elementary school|public school).*' and ZI_C_COMPANY_REVENUE>150) a
left join (select * from zoominfo.brick.zi_company_contact_combined where ZoomInfo_Contact_ID is not null and ((zi_c_num_locations=1 and zi_c_employees<=10) or 
(zi_c_num_locations>1 and zi_c_num_locations<=50 and zi_c_employees>10)) and country in ('United States') and Contact_Accuracy_Score > 80 
-- and ZI_C_SIC4 in ('5611','5621','5631','5641','5651','5655','5661','5697','5944','5844','5309','5310','5311','5399','5331','5712','5714','5719','5137','5733','5735','5931','5932','5977','5943','5945','5947','5948','5950','5970','5971','5973','5651','5995','5999','5912')
) b ON b.ZI_C_COMPANY_ID = a.ZI_C_COMPANY_ID
join REFERENCE.MARKETING.ZI_TOPIC_PRODUCT_MAPPING c on a.topic=c.topic
where  (email not in (select distinct IFNULL(email,'NA') from bi.marketing.intent_campaign))
and email not regexp '.*(.gov|.edu).*' and c.product in ('Retail','Restaurant & Retail')
and ((Phone is not null or mobile is not null) and (Phone!='' or mobile is not null))
and not exists (select 1 from REFERENCE.MARKETING.ZI_JOB_EXCLUSION_LIST where lower(b.job_title) like lower(job_title)|| '%')
qualify row_number() over (partition by b.ZoomInfo_Contact_ID order by a.SIGNAL_SCORE) = 1
and row_number() over (partition by b.ZI_C_LOCATION_ID order by b.ZI_C_LOCATION_ID) <= 5
and row_number() over (partition by phone order by Job_Title_Hierarchy_Level desc) = 1
);

In [ ]:
insert into bi.marketing.intent_campaign (
select  
b.ZI_C_LOCATION_ID,
b.ZI_C_COMPANY_ID,
a.ZI_C_COMPANY_NAME,
b.ZI_C_STREET,
b.ZI_C_STREET_2,
b.ZI_C_CITY,
b.ZI_C_STATE,
b.ZI_C_ZIP,
b.ZI_C_COUNTRY,
b.ZI_ES_LOCATION_ID,
b.ZI_C_URL,
b.ZI_C_LATITUDE,
b.ZI_C_LONGITUDE,
b.ZI_C_EMPLOYEES,
b.ZI_C_REVENUE_RANGE,
b.ZI_C_REVENUE,
coalesce(d."Mobile phone",d."Direct Phone Number",b.ZI_C_COMPANY_PHONE,b.ZI_C_PHONE) as phone,
b.ZI_C_INDUSTRY_PRIMARY,
b.ZI_C_SUB_INDUSTRY_PRIMARY,
b.ZI_C_SIC4,
b.ZI_C_NAICS6,
b.ZI_C_COMPANY_REVENUE_RANGE,
b.ZI_C_LINKEDIN_URL,
b.ZI_C_FACEBOOK_URL,
b.ZI_C_TWITTER_URL,
b.ZI_C_YELP_URL,
b.ZI_C_EIN,
b.ZI_C_NUM_LOCATIONS,
b.ZI_C_HR_CONTACTS,
b.ZI_C_SALES_CONTACTS,
b.ZI_C_MARKETING_CONTACTS,
b.ZI_C_FINANCE_CONTACTS,
b.ZI_C_C_SUITE_CONTACTS,
b.ZI_C_ENGINEERING_CONTACTS,
b.ZI_C_IT_CONTACTS,
b.ZI_C_OPERATIONS_CONTACTS,
b.ZI_C_TECH_IDS,
d."ZoomInfo Contact ID" as ZoomInfo_Contact_ID,
d."Last Name" as Last_Name,
d."First Name" as First_Name,
d."Middle Name" as Middle_Name,
d."Suffix" as Suffix,
d."Job Title" as Job_Title,
d."Job Title Hierarchy Level" as Job_Title_Hierarchy_Level,
d."Job Function" as Job_Function,
d."Management Level" as Management_Level,
d."Company Division Name" as Company_Division_Name,
coalesce(d."Email Address",d."Supplemental Email") as email,
d."Email Domain" as Email_Domain,
d."Department" as Department,
d."Contact Accuracy Score" as Contact_Accuracy_Score,
d."Contact Accuracy Grade" as Contact_Accuracy_Grade,
d."ZoomInfo Contact Profile URL" as ZoomInfo_Contact_Profile_URL,
d."LinkedIn Contact Profile URL" as LinkedIn_Contact_Profile_URL,
d."Notice Provided Date" as Notice_Provided_Date,
d."Person Street" as Person_Street,
d."Person City" as Person_City,
d."Person State" as Person_State,
d."Person Zip Code" as Person_Zip_Code,
d."Country" as Country,
d."ZoomInfo Company ID" as ZoomInfo_Company_ID,
a.INTENT_ID,
a.DATE,
a.TOPIC,
a.CATEGORY,
a.SIGNAL_SCORE,
a.AUDIENCE_STRENGTH,
a.SIGNAL_LOCATION,
a.ZI_C_COMPANY_REVENUE,
'Restaurant' as product,
current_date as dtecreated,
d."Mobile phone" as mobile
from (select * from zoominfo_rw_secondary.zoominfo.zi_intent_company_data where  date>CURRENT_DATE()-7
and ZI_C_COMPANY_REVENUE>150 and lower(ZI_C_COMPANY_NAME) not regexp '.*(supply|construction).*') a
left join (select * from zoominfo_rw_secondary.zoominfo.zi_comp_with_additional_insights_with_linkages_20240101_us_only where ((zi_c_num_locations=1 and zi_c_employees<=75) or (zi_c_num_locations>1 and zi_c_num_locations<=250 and zi_c_employees>10))  
-- and ZI_C_NAICS6 in ('722410','713950','722515','722511','722513') 
and  ZI_C_INDUSTRY_PRIMARY not in ('Construction')  and ZI_C_INDUSTRY_PRIMARY!='Retail'
) b ON b.ZI_C_COMPANY_ID = a.ZI_C_COMPANY_ID
left join (select * from zoominfo_rw_secondary.zoominfo.zi_contact_databrick_premium_20231201 
where "ZoomInfo Contact ID" is not null and "Department" not in ('Medical & Health') and "Country" in ('United States','Canada') and "Contact Accuracy Score" > 80) d
on d."ZoomInfo Company ID" = a.ZI_C_COMPANY_ID
join REFERENCE.MARKETING.ZI_TOPIC_PRODUCT_MAPPING c on a.topic=c.topic
where  (email not in (select distinct IFNULL(email,'NA') from bi.marketing.intent_campaign))
and email not regexp '.*(.gov|.edu|.org).*'
and c.product in ('Restaurant POS')
and ((Phone is not null or mobile is not null) and (Phone!='' or mobile is not null))
and not exists (select 1 from REFERENCE.MARKETING.ZI_JOB_EXCLUSION_LIST where lower(d."Job Title") like lower(job_title)|| '%')
qualify row_number() over (partition by d."ZoomInfo Contact ID" order by a.SIGNAL_SCORE) = 1
and row_number() over (partition by b.ZI_C_LOCATION_ID order by b.ZI_C_LOCATION_ID) <= 5
and row_number() over (partition by phone order by Job_Title_Hierarchy_Level desc) = 1
) ;

In [ ]:
def clean_phone_number(phone_str):
    try:
        phone = phonenumbers.parse(phone_str,"US")
        if phone.country_code!=1:
            return None,None
        
        national_number=str(phone.national_number)
        extension_number=phone.extension if phone.extension else None

        return national_number,extension_number
    except phonenumbers.NumberParseException:
        return None,None

In [ ]:
select distinct *, 'Manager' as management
from bi.marketing.intent_campaign where dtecreated=current_date and phone is not null and phone != '' 
and ZI_C_NAME!='' and ZI_C_NAME is not null and email is not null 
and Management_Level is not null and Management_Level != 'Non-Manager' and email != ''
union
select distinct *, 'Non-Manager' as management
from bi.marketing.intent_campaign where dtecreated=current_date
and phone is not null and phone != '' and email is not null and Management_Level = 'Non-Manager' 
and email != '' and ZI_C_NAME!='' and ZI_C_NAME is not null;

In [ ]:
zi_df = cell2.to_pandas()

In [ ]:
zi_df = zi_df.replace("",None)
zi_df[['Mobile_clean','Mobile_Extension']]=zi_df['MOBILE'].apply(lambda x:pd.Series(clean_phone_number(x)))
zi_df=zi_df.rename(columns={'Mobile_clean':'Mobile'})
zi_df['Mobile']=zi_df.Mobile.where((zi_df['Mobile'].astype('str').map(len)<=10))
zi_df=zi_df.drop(columns=['Mobile_Extension','MOBILE'])
zi_df[['PHONE_clean','Phone_Extension']]=zi_df['PHONE'].apply(lambda x:pd.Series(clean_phone_number(x)))
zi_df=zi_df.rename(columns={'PHONE_clean':'Phone','Phone_Extension':'Phone Extension'})
zi_df=zi_df.dropna(subset=["Phone"]) 
zi_df=zi_df.drop(columns=['PHONE'])
zi_df=zi_df.drop(zi_df[(zi_df['Phone'].map(len)!=10)].index)

In [ ]:
zi_df=zi_df.rename(columns={'ZI_C_NAME':'Company','ZI_C_STREET':'Address 1','ZI_C_STREET_2':'Address 2','ZI_C_CITY':'City','ZI_C_STATE':'State','ZI_C_ZIP':'Zip','ZI_C_COUNTRY':'Country','ZI_C_LOCATION_ID':'ZI_Location_ID','ZI_C_EMPLOYEES':'No. of Employees','ZI_C_REVENUE':'Annual Revenue/Company Sales','JOB_TITLE':'Title','FIRST_NAME':'First name','LAST_NAME':'Last name','EMAIL':'Email','DEPARTMENT':'Department','TOPIC':'Zi_Intent_Topic','ZI_C_INDUSTRY_PRIMARY':'Industry','ZI_C_SIC4':'SIC Code','ZOOMINFO_CONTACT_ID':'ZI_Contact_ID','ZI_C_NUM_LOCATIONS':'No of Locations','ZI_C_URL':'Company URL'})
zi_df=zi_df[['Company','Address 1','Address 2','City','State','Zip','Country','ZI_Location_ID','No. of Employees','Annual Revenue/Company Sales','Industry','SIC Code','Phone','No of Locations','ZI_Contact_ID','Title','First name','Last name','Email','Department','Zi_Intent_Topic','ZOOMINFO_COMPANY_ID','Mobile','Phone Extension','Company URL','PRODUCT','MANAGEMENT','DTECREATED']]
zi_df=zi_df.reset_index(drop=True)

In [ ]:
if len(zi_df.index)>0:
    session.write_pandas(zi_df,"ZI_FORMATTING_WEEKLY_INTENT",database="REFERENCE",schema="MARKETING",auto_create_table=False,overwrite=False)
else:
    pass

In [ ]:
-- Verification
select count(1) as numrows from REFERENCE.MARKETING.ZI_FORMATTING_WEEKLY_INTENT where dtecreated=current_date;

In [ ]:
count_df=cell16.to_pandas()
if len(zi_df.index)==count_df['NUMROWS'][0]:
    print(f"Count matches. Write Successful. ZI_DF len: {len(zi_df.index)}. COUNT_DF len: {count_df['NUMROWS'][0]}")
    cc_email = ''
    sub1  = "ZI Data laoded on "+ pd.Timestamp("today").strftime("%Y-%m-%d") + ""
    body1 = f"Table REFERENCE.MARKETING.ZI_FORMATTING_WEEKLY_INTENT is reloaded on "+ pd.Timestamp("today").strftime("%Y-%m-%d") + f"."
    message = Mail(
    from_email='bi@heartland.us',
    to_emails=[To('arvindraj.rajakumar@e-hps.com')],
    subject=sub1,
    plain_text_content=body1
    )
    if cc_email:
        message.add_cc(cc_email)

    sg = SendGridAPIClient(sendgrid_api_key)
    response = sg.send(message)
else:
    print(f"Count did not match. ZI_DF len: {len(zi_df.index)}. COUNT_DF len: {count_df['NUMROWS'][0]}")
    cc_email = ''
            
    sub1  = "ZI Data laoded on "+ pd.Timestamp("today").strftime("%Y-%m-%d") + " - Count mismatch"
    body1 = f"Table REFERENCE.MARKETING.ZI_FORMATTING_WEEKLY_INTENT is reloaded on "+ pd.Timestamp("today").strftime("%Y-%m-%d") + f". It had a count mismatch between the dataframe and the table. Please check the latest execution of the notebook."
    message = Mail(
    from_email='bi@heartland.us',
    to_emails=[To('arvindraj.rajakumar@e-hps.com')],
    subject=sub1,
    plain_text_content=body1
    )
    if cc_email:
        message.add_cc(cc_email)

    sg = SendGridAPIClient(sendgrid_api_key)
    response = sg.send(message)